In [1]:
import random

from collections import defaultdict

In [2]:
N = 2**16 - 1
N

65535

# Cyclotomic cosets modulo $N$ over $GF(2)$

In [3]:
cyclotomic_cosets = []
used = [False for _ in range(N)]
for s in range(N):
    if used[s]:
        continue
    used[s] = True
    coset = [s]
    cur_elem = (s * 2) % N
    while cur_elem != s:
        coset.append(cur_elem)
        used[cur_elem] = True
        cur_elem = (cur_elem * 2) % N
    # coset.sort()
    cyclotomic_cosets.append(coset)
cyclotomic_cosets[:3]

[[0],
 [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768],
 [3,
  6,
  12,
  24,
  48,
  96,
  192,
  384,
  768,
  1536,
  3072,
  6144,
  12288,
  24576,
  49152,
  32769]]

In [4]:
cyclotomic_coset_size_cnt = defaultdict(int)
cyclotomic_cosets_by_size = defaultdict(list)
for coset in cyclotomic_cosets:
    cyclotomic_coset_size_cnt[len(coset)] += 1
    cyclotomic_cosets_by_size[len(coset)].append(coset)

sum_cnt = 0
for k, v in sorted(cyclotomic_coset_size_cnt.items()):
    sum_cnt += k * v
    print(k, ":", v)
sum_cnt

1 : 1
2 : 1
4 : 3
8 : 30
16 : 4080


65535

In [5]:
threshold_by_size = {
    1: 0,
    2: 1,
    4: 3,
    8: 15,
    16: 255,
}

# Field $GF(2^{16})$

In [6]:
# F.<a> = GF(2^16, modulus="primitive")
# F

In [7]:
_F.<t> = GF(2)[]
F.<a> = GF(2^16, modulus=(t^16 + t^5 + t^3 + t^2 + 1))
F

Finite Field in a of size 2^16

In [8]:
for i in range(1, N + 1):
    if a^i == 1:
        print(i)

65535


In [9]:
R.<x> = F[]
R

Univariate Polynomial Ring in x over Finite Field in a of size 2^16

# Reed-Solomon code over $GF(2^{16})$

In [10]:
def generate_payload_data_vector_f(k):
    payload_data_vector = [F.random_element() for _ in range(k)]
    payload_data_vector = [e if e != 0 else a for e in payload_data_vector]
    return payload_data_vector

In [11]:
def erase_symbols_f(code_vector, positions, t):
    erased_symbols = sorted(random.sample(range(len(code_vector)), t))
    assert len(set(erased_symbols)) == t
    y = code_vector.copy()
    erased_positions = []
    for i in erased_symbols:
        y[i] = 0
        erased_positions.append(positions[i])
    return y, erased_symbols, erased_positions

## Encoding

### Step-by-step

In [12]:
k = 100
r = 43
payload_data_vector = generate_payload_data_vector_f(k)

payload_data_vector[:10]

[a^15 + a^13 + a^11 + a^7 + a^5 + a^3 + a^2 + a + 1,
 a^13 + a^12 + a^8 + a^5 + a^3 + a + 1,
 a^14 + a^13 + a^12 + a^11 + a^10 + a^9 + a^8 + a^7 + a^3 + a^2 + a + 1,
 a^14 + a^13 + a^12 + a^11 + a^10 + a^8 + a^4 + a^3 + a,
 a^14 + a^12 + a^11 + a^10 + a^5 + a^2 + 1,
 a^13 + a^10 + a^9 + a^6 + a^5 + a^4 + a^2 + a + 1,
 a^15 + a^14 + a^12 + a^6 + a^5 + a^3,
 a^14 + a^13 + a^7 + a^4 + a^3 + 1,
 a^15 + a^14 + a^13 + a^12 + a^9 + a^8 + a^7 + a^5 + a,
 a^15 + a^13 + a^12 + a^10 + a^9 + a^8 + a^7 + a^6 + a^3 + a^2 + a]

In [13]:
def get_positions_f(k, r):
    assert k + r <= N
    
    inf_positions = []
    par_positions = []
    
    local_threshold = {k: v for k, v in threshold_by_size.items()}
    idx_by_size = defaultdict(int)
    for size, cosets in sorted(cyclotomic_cosets_by_size.items(), reverse=True):
        for coset in cosets:
            if r <= threshold_by_size[size]:
                break
            r -= size
            par_positions.extend(coset)
            
            idx_by_size[size] += 1
    assert r == 0
    
    for i in range(4):
        for idx in range(i + 1, 5):
            local_threshold[1 << idx] -= idx_by_size[1 << i] << i
    
    # inf_positions = [i for i in range(N) if not used_positions[i]][:k]

    for size, cosets in sorted(cyclotomic_cosets_by_size.items(), reverse=True):
        for coset in cosets[idx_by_size[size]:]:
            if k <= local_threshold[size]:
                break
            inf_positions.extend(coset[:k])
            k -= min(k, size)
    assert k == 0
    
    # inf_positions.sort()
    # par_positions.sort()
    return inf_positions, par_positions

inf_positions, par_positions = get_positions_f(k, r)
inf_positions[:10], par_positions[:10]

([2313, 4626, 9252, 18504, 37008, 8481, 16962, 33924, 2827, 5654],
 [257, 514, 1028, 2056, 4112, 8224, 16448, 32896, 771, 1542])

In [14]:
def get_check_symbol_locator_poly_f(par_positions):
    check_symbol_locator_poly = 1
    for par_pos in par_positions:
        check_symbol_locator_poly *= 1 - (a^par_pos) * x
    return check_symbol_locator_poly

check_symbol_locator_poly = get_check_symbol_locator_poly_f(par_positions)
check_symbol_locator_poly

x^43 + x^41 + x^36 + x^34 + x^33 + x^32 + x^31 + x^29 + x^28 + x^26 + x^25 + x^24 + x^23 + x^22 + x^19 + x^17 + x^15 + x^14 + x^12 + x^10 + x^7 + x^4 + x^2 + 1

In [15]:
def get_forney_coefficients_f(check_symbol_locator_poly, par_positions):
    check_symbol_locator_poly_derivative = derivative(check_symbol_locator_poly)
    forney_coefficients = [(a^par_pos) / check_symbol_locator_poly_derivative(a^(-par_pos)) for par_pos in par_positions]
    return forney_coefficients
    
forney_coefficients = get_forney_coefficients_f(check_symbol_locator_poly, par_positions)
forney_coefficients[:10]

[a^15 + a^14 + a^13 + a^11 + a^10 + a^9 + a^8 + a^7 + a^6 + a^3 + a^2,
 a^14 + a^13 + a^12 + a^11 + a^10 + a^4 + a^3 + a^2,
 a^14 + a^8 + a^7 + a^4 + a^3 + a,
 a^15 + a^12 + a^8 + a^5 + a^4 + a + 1,
 a^14 + a^13 + a^11 + a^8 + a^5 + a^4 + a^2 + a + 1,
 a^14 + a^13 + a^11 + a^9 + a^5 + a,
 a^14 + a^13 + a^11 + a^9 + a^8 + a^7 + a^5 + a^3 + a,
 a^13 + a^11 + a^9 + a^8 + a^7 + a^6 + a^2 + a + 1,
 a^15 + a^13 + a^11 + a^10 + a^9 + a^8 + a^7 + a^6 + a^4 + a^3 + a^2 + a,
 a^15 + a^13 + a^11 + a^10 + a^8 + a^6 + a]

In [16]:
def get_payload_data_vector_syndrome_poly_f(payload_data_vector, inf_positions, r, verbose=False):
    k = len(payload_data_vector)
    payload_data_vector_syndrome_poly = 0
    for j in range(r):
        if verbose:
            print(">> j = ", j)
        
        s_j = 0
        for i in range(k):
            if verbose:
                print(">> s_j +=", (payload_data_vector[i] * (a^(inf_positions[i] * j))).to_integer(), end=" ")
            
            s_j += payload_data_vector[i] * (a^(inf_positions[i] * j))
            
            if verbose:
                print("|", "s_j =", s_j.to_integer())
        
        payload_data_vector_syndrome_poly += s_j * (x^j)
        
    return payload_data_vector_syndrome_poly
    
payload_data_vector_syndrome_poly = get_payload_data_vector_syndrome_poly_f(payload_data_vector, inf_positions, r)
payload_data_vector_syndrome_poly

(a^14 + a^13 + a^7 + a^4 + a^3 + a^2 + a)*x^42 + (a^14 + a^13 + a^11 + a^10 + a^8 + a^7 + a^5 + a^4 + a^3 + a^2 + a)*x^41 + (a^15 + a^14 + a^12 + a^8 + a^6 + a^3 + 1)*x^40 + (a^15 + a^13 + a^12 + a^10 + a^8 + a^7 + a^4 + a^3 + a^2)*x^39 + (a^14 + a^13 + a^12 + a^11 + a^8 + a^7 + a^6 + a^4 + a^3 + a)*x^38 + (a^15 + a^13 + a^10 + a^8 + a^7)*x^37 + (a^15 + a^14 + a^11 + a^10 + a^6)*x^36 + (a^14 + a^12 + a^11 + a^9 + a^7 + a^6 + a^5)*x^35 + (a^14 + a^10 + a^9 + a^6 + a^5 + a^4 + a + 1)*x^34 + (a^15 + a^14 + a^12 + a^11 + a^9 + a^4 + a^2 + a)*x^33 + (a^14 + a^13 + a^12 + a^11 + a^8 + a^4 + 1)*x^32 + (a^11 + a^8 + a^7 + a^6 + a^3 + a^2)*x^31 + (a^15 + a^14 + a^13 + a^12 + a^11 + a^10 + a^9 + a^5 + 1)*x^30 + (a^12 + a^8 + a^7 + a^6 + a^5 + a^4 + a^3 + a + 1)*x^29 + (a^15 + a^13 + a^12 + a^8 + a^7 + a^6 + a^5 + a^4 + a^2 + a + 1)*x^28 + (a^10 + a^8 + a^7 + a^5 + a)*x^27 + (a^14 + a^11 + a^8 + a^4 + a^3)*x^26 + (a^15 + a^14 + a^12 + a^10 + a^7 + a^6 + a^5 + a^3 + 1)*x^25 + (a^15 + a^13 + a^12 +

In [17]:
def get_check_symbol_evaluator_poly_f(check_symbol_locator_poly, payload_data_vector_syndrome_poly, r):
    check_symbol_evaluator_poly = (check_symbol_locator_poly * payload_data_vector_syndrome_poly) % (x^r)
    return check_symbol_evaluator_poly

check_symbol_evaluator_poly = get_check_symbol_evaluator_poly_f(check_symbol_locator_poly, payload_data_vector_syndrome_poly, r)
check_symbol_evaluator_poly

(a^15 + a^13 + a^3)*x^42 + (a^15 + a^14 + a^13 + a^12 + a^10 + a^9 + a^8 + a^7 + a^6 + a^4 + a^3 + a + 1)*x^41 + (a^15 + a^8 + a^7 + a^3 + a + 1)*x^40 + (a^14 + a^12 + a^11 + a^10 + a^8 + a^6 + a^5 + a^4 + a^2)*x^39 + (a^14 + a^10 + a^9 + a^7 + a^3)*x^38 + (a^13 + a^11 + a^10 + a^9 + a^6 + a^4 + a^2)*x^37 + (a^12 + a^9 + a^7 + a^6 + a^4 + a^3 + a^2 + a)*x^36 + (a^14 + a^13 + a^11 + a^10 + a^8 + a^6 + a^2 + a)*x^35 + (a^11 + a^10 + a^8 + a^7 + a^6 + a^5 + a^4 + a^2)*x^34 + (a^15 + a^11 + a^7 + a^5 + a^2 + a)*x^33 + (a^15 + a^13 + a^12 + a^10 + a^8 + a^6 + a^5 + a^4 + a^3 + 1)*x^32 + (a^15 + a^14 + a^13 + a^11 + a^10 + a^6 + a^5 + a)*x^31 + (a^12 + a^9 + a^8 + a^6 + a^2 + a)*x^30 + (a^15 + a^10 + a^9 + a^8 + a^7 + a^5 + a^2)*x^29 + (a^15 + a^13 + a^12 + a^10 + a^7 + a^3)*x^28 + (a^14 + a^10 + a^7 + a^6 + a^5 + a^4 + a^3 + a^2)*x^27 + (a^13 + a^12 + a^11 + a^10 + a^9 + a^8 + a^7 + a^5 + a^3 + a)*x^26 + (a^15 + a^13 + a^12 + a^11 + a^10 + a^8 + a^7 + a^6 + 1)*x^25 + (a^15 + a^14 + a^12 + a

In [18]:
def get_check_symbols_f(forney_coefficients, check_symbol_evaluator_poly, par_positions):
    r = len(par_positions)
    check_symbols = [forney_coefficients[i] * check_symbol_evaluator_poly(a^(-par_positions[i])) for i in range(r)]
    return check_symbols

check_symbols = get_check_symbols_f(forney_coefficients, check_symbol_evaluator_poly, par_positions)
check_symbols[:10]

[a^13 + a^12 + a^11 + a^7 + a^5 + a^4 + a^3 + a,
 a^15 + a^13 + a^12 + a^11 + a^10 + a^8 + a^7 + a^4 + a^3 + 1,
 a^14 + a^13 + a^12 + a^11 + a^10 + a^9 + a^7 + a^6 + a^3 + a^2 + 1,
 a^15 + a^13 + a^12 + a^11 + a^10 + a^9 + a^6 + a^4 + a,
 a^15 + a^14 + a^11 + a^9 + a^5 + a^4 + a^2,
 a^15 + a^13 + a^12 + a^11 + a^10 + a^8 + a^6 + a^5 + 1,
 a^15 + a^14 + a^12 + a^11 + a^10 + a^9 + a^7 + a^5 + 1,
 a^15 + a^13 + a^12 + a^8 + a^7 + a^6 + a^2 + a + 1,
 a^13 + a^12 + a^9 + a^8 + a^6 + a^5 + a^4 + a + 1,
 a^15 + a^14 + a^10 + a^5 + a^4]

In [19]:
def get_code_vector_f(payload_data_vector, check_symbols):
    code_vector = payload_data_vector + check_symbols
    return code_vector

code_vector = get_code_vector_f(payload_data_vector, check_symbols)
positions = inf_positions + par_positions
print(code_vector[:10], "...", code_vector[k:k+10], "...")

[a^15 + a^13 + a^11 + a^7 + a^5 + a^3 + a^2 + a + 1, a^13 + a^12 + a^8 + a^5 + a^3 + a + 1, a^14 + a^13 + a^12 + a^11 + a^10 + a^9 + a^8 + a^7 + a^3 + a^2 + a + 1, a^14 + a^13 + a^12 + a^11 + a^10 + a^8 + a^4 + a^3 + a, a^14 + a^12 + a^11 + a^10 + a^5 + a^2 + 1, a^13 + a^10 + a^9 + a^6 + a^5 + a^4 + a^2 + a + 1, a^15 + a^14 + a^12 + a^6 + a^5 + a^3, a^14 + a^13 + a^7 + a^4 + a^3 + 1, a^15 + a^14 + a^13 + a^12 + a^9 + a^8 + a^7 + a^5 + a, a^15 + a^13 + a^12 + a^10 + a^9 + a^8 + a^7 + a^6 + a^3 + a^2 + a] ... [a^13 + a^12 + a^11 + a^7 + a^5 + a^4 + a^3 + a, a^15 + a^13 + a^12 + a^11 + a^10 + a^8 + a^7 + a^4 + a^3 + 1, a^14 + a^13 + a^12 + a^11 + a^10 + a^9 + a^7 + a^6 + a^3 + a^2 + 1, a^15 + a^13 + a^12 + a^11 + a^10 + a^9 + a^6 + a^4 + a, a^15 + a^14 + a^11 + a^9 + a^5 + a^4 + a^2, a^15 + a^13 + a^12 + a^11 + a^10 + a^8 + a^6 + a^5 + 1, a^15 + a^14 + a^12 + a^11 + a^10 + a^9 + a^7 + a^5 + 1, a^15 + a^13 + a^12 + a^8 + a^7 + a^6 + a^2 + a + 1, a^13 + a^12 + a^9 + a^8 + a^6 + a^5 + a^4 + 

### Encoding function

In [20]:
def encode_f(payload_data_vector, r, verbose=False):
    k = len(payload_data_vector)
    inf_positions, par_positions = get_positions_f(k, r)
    positions = inf_positions + par_positions
    
    check_symbol_locator_poly = get_check_symbol_locator_poly_f(par_positions)
    forney_coefficients = get_forney_coefficients_f(check_symbol_locator_poly, par_positions)
    payload_data_vector_syndrome_poly = get_payload_data_vector_syndrome_poly_f(payload_data_vector, inf_positions, r, verbose=verbose)
    check_symbol_evaluator_poly = get_check_symbol_evaluator_poly_f(check_symbol_locator_poly, payload_data_vector_syndrome_poly, r)
    
    check_symbols = get_check_symbols_f(forney_coefficients, check_symbol_evaluator_poly, par_positions)
    code_vector = get_code_vector_f(payload_data_vector, check_symbols)
    
    if verbose:
        def p_to_l(p):
            return [c.to_integer() for c in p.coefficients(sparse=False)]
        
        print("> payload_data_vector:", [c.to_integer() for c in payload_data_vector])
        print("> inf_positions:", inf_positions)
        print("> par_positions:", par_positions)
        print("> check_symbol_locator_poly:", p_to_l(check_symbol_locator_poly))
        print("> forney_coefficients:", [c.to_integer() for c in forney_coefficients])
        print("> payload_data_vector_syndrome_poly:", p_to_l(payload_data_vector_syndrome_poly))
        print("> check_symbol_evaluator_poly:", p_to_l(check_symbol_evaluator_poly))
        print("> check_symbols:", [c.to_integer() for c in check_symbols])
    
    return code_vector, positions

## Erasure correction

### Step-by-step

In [21]:
t = 43
y, erased_symbols, erased_positions = erase_symbols_f(code_vector, positions, t)

y[:10], erased_symbols[:10], erased_positions[:10]

([0,
  a^13 + a^12 + a^8 + a^5 + a^3 + a + 1,
  a^14 + a^13 + a^12 + a^11 + a^10 + a^9 + a^8 + a^7 + a^3 + a^2 + a + 1,
  0,
  a^14 + a^12 + a^11 + a^10 + a^5 + a^2 + 1,
  0,
  a^15 + a^14 + a^12 + a^6 + a^5 + a^3,
  a^14 + a^13 + a^7 + a^4 + a^3 + 1,
  0,
  a^15 + a^13 + a^12 + a^10 + a^9 + a^8 + a^7 + a^6 + a^3 + a^2 + a],
 [0, 3, 5, 8, 17, 19, 20, 24, 27, 29],
 [2313, 18504, 8481, 2827, 6682, 26728, 53456, 3855, 30840, 57825])

In [22]:
def get_syndrome_poly_f(y, positions, t):
    n = len(positions)
    syndrome_poly = 0
    for i in range(t):
        s_i = sum(y[j] * (a^(positions[j] * i)) for j in range(n))
        syndrome_poly += s_i * (x^i)
    return syndrome_poly

syndrome_poly = get_syndrome_poly_f(y, positions, t)
syndrome_poly

(a^14 + a^12 + a^11 + a^9 + a^8 + a^7 + a^6 + a^5 + a^4 + a^2 + a + 1)*x^42 + (a^15 + a^14 + a^13 + a^11 + a^10 + a^5 + a^4 + a^3 + a^2 + a)*x^41 + (a^12 + a^9 + a^8 + a^7 + a^5 + a^4 + a^3 + a^2)*x^40 + (a^15 + a^14 + a^13 + a^12 + a^11 + a^9 + a^8 + a^7 + a^6 + a^5 + a^2 + 1)*x^39 + (a^13 + a^12 + a^10 + a^7 + a^4 + a)*x^38 + (a^15 + a^13 + a^11 + a^10 + a^9 + a^6 + a^4 + a^2 + a)*x^37 + (a^15 + a^14 + a^13 + a^11 + a^7 + a^6 + a^5 + a^4 + a^3)*x^36 + (a^15 + a^14 + a^11 + a^5 + a^4 + a^3 + a + 1)*x^35 + (a^14 + a^11 + a^10 + a^8 + a^5 + a^4 + a^3 + a^2)*x^34 + (a^14 + a^13 + a^10 + a^9 + a^8 + a^6 + a^3 + 1)*x^33 + (a^15 + a^14 + a^12 + a^11 + a^10 + a^7 + a^6 + a^5 + a^4 + a^3 + 1)*x^32 + (a^15 + a^14 + a^11 + a^8 + a^7 + a^6 + a^5 + a + 1)*x^31 + (a^15 + a^14 + a^10 + a^8 + a^7 + a^5 + a^4 + a^3 + 1)*x^30 + (a^14 + a^13 + a^10 + a^9 + a^8 + a^7 + a^6 + a^5 + a^4 + a^3)*x^29 + (a^13 + a^12 + a^11 + a^9 + a^8 + a^7 + a^6 + a^5 + a^4 + a^3 + a + 1)*x^28 + (a^13 + a^11 + a^10 + a^9 + 

In [23]:
def get_erasure_locator_poly_f(erased_positions):
    erasure_locator_poly = 1
    for erased_pos in erased_positions:
        erasure_locator_poly *= 1 - (a^erased_pos) * x
    return erasure_locator_poly

erasure_locator_poly = get_erasure_locator_poly_f(erased_positions)
erasure_locator_poly

(a^15 + a^12 + a^9 + a^6 + a^4 + a^3 + 1)*x^43 + (a^15 + a^13 + a^12 + a^11 + a^9 + a^8 + a^6 + a^5 + a^4 + a^3 + a^2 + 1)*x^42 + (a^15 + a^13 + a^12 + a^11 + a^9 + a^8 + a^6 + a^5 + a^4 + a^3 + a^2 + 1)*x^41 + (a^14 + a^13 + a^11 + a^9 + a^8 + a^7 + a^5 + a^3 + a)*x^40 + (a^14 + a^12 + a^10 + a^7 + a^6 + a^3 + a^2 + a + 1)*x^39 + (a^8 + a^7 + a^3 + 1)*x^38 + (a^12 + a^10 + a^9 + a^8 + a^7 + a^6 + a^3)*x^37 + (a^15 + a^14 + a^10 + a^8 + a^7 + a^6 + a^5 + a^4 + a^3 + a^2)*x^36 + (a^14 + a^9 + a^8 + a^6 + a^5 + a^4 + a^3)*x^35 + (a^15 + a^14 + a^12 + a^7 + a^6 + a^4 + a^2 + a)*x^34 + (a^14 + a^13 + a^11 + a^8 + a^6 + a^3 + 1)*x^33 + (a^14 + a^4 + a + 1)*x^32 + (a^12 + a^10 + a^9 + a^5 + a^4 + a^3 + a^2 + a + 1)*x^31 + (a^15 + a^14 + a^13 + a^12 + a^11 + a^9 + a^7 + a^6 + a^5 + a^2 + a)*x^30 + (a^15 + a^14 + a^13 + a^12 + a^11 + a^9 + a^7 + a^4 + a^3)*x^29 + (a^15 + a^14 + a^10 + a^9 + a^7 + a^4 + a^2 + a)*x^28 + (a^15 + a^14 + a^12 + a^8 + a^6 + a^4 + a^3 + a^2 + a + 1)*x^27 + (a^15 + a^

In [24]:
def get_erasure_evaluator_poly_f(erasure_locator_poly, syndrome_poly, t):
    erasure_evaluator_poly = (erasure_locator_poly * syndrome_poly) % (x^t)
    return erasure_evaluator_poly

erasure_evaluator_poly = get_erasure_evaluator_poly_f(erasure_locator_poly, syndrome_poly, t)
erasure_evaluator_poly

(a^15 + a^13 + a^9 + a^8 + a^7 + a^4 + a^3 + 1)*x^42 + (a^11 + a^9 + a^5 + a^4 + a^3 + a^2)*x^41 + (a^14 + a^13 + a^12 + a^10 + a^9 + a^7 + a^2)*x^40 + (a^13 + a^12 + a^11 + a^9 + a^8 + a^7 + a^6 + a^2 + a + 1)*x^39 + (a^15 + a^13 + a^12 + a^10 + a^6 + a^4 + a^3 + 1)*x^38 + (a^12 + a^11 + a^8 + a^5 + a^4 + a^3 + 1)*x^37 + (a^11 + a^10 + a^9 + a + 1)*x^36 + (a^15 + a^14 + a^11 + a^9 + a^7 + a^6 + 1)*x^35 + (a^12 + a^11 + a^9 + a^7 + a^6 + a^5 + a^4 + a^3 + a^2 + 1)*x^34 + (a^10 + a^8 + a^6 + a^5 + a + 1)*x^33 + (a^15 + a^13 + a^12 + a^11 + a^10 + a^9 + a^6 + a^3 + a + 1)*x^32 + (a^12 + a^11 + a^9 + a^6 + a^5 + a + 1)*x^31 + (a^13 + a^12 + a^11 + a^9 + a^7 + a^6 + a^3)*x^30 + (a^15 + a^14 + a^12 + a^11 + a^10 + a^8 + a^6 + a^5 + a^4 + a)*x^29 + (a^15 + a^14 + a^13 + a^10 + a^8 + a^7 + a^5 + a^3 + 1)*x^28 + (a^15 + a^14 + a^10 + a^9 + a^8 + a^7 + a^6 + a^5 + a^3 + a^2)*x^27 + (a^14 + a^13 + a^11 + a^10 + a^9 + a^6 + a^4 + a + 1)*x^26 + (a^14 + a^13 + a^10 + a^9 + a^8 + a^7 + a^4 + a)*x^25

In [25]:
def get_repaired_code_vector_f(y, erasure_locator_poly, erasure_evaluator_poly, erased_symbols, erased_positions):
    repaired_code_vector = y.copy()
    erasure_locator_poly_derivative = derivative(erasure_locator_poly)
    for i, erased_pos in zip(erased_symbols, erased_positions):
        repaired_code_vector[i] = (a^erased_pos) * erasure_evaluator_poly(a^(-erased_pos)) / erasure_locator_poly_derivative(a^(-erased_pos))
    return repaired_code_vector
    
repaired_code_vector = get_repaired_code_vector_f(y, erasure_locator_poly, erasure_evaluator_poly, erased_symbols, erased_positions)
repaired_code_vector[:10]

[a^15 + a^13 + a^11 + a^7 + a^5 + a^3 + a^2 + a + 1,
 a^13 + a^12 + a^8 + a^5 + a^3 + a + 1,
 a^14 + a^13 + a^12 + a^11 + a^10 + a^9 + a^8 + a^7 + a^3 + a^2 + a + 1,
 a^14 + a^13 + a^12 + a^11 + a^10 + a^8 + a^4 + a^3 + a,
 a^14 + a^12 + a^11 + a^10 + a^5 + a^2 + 1,
 a^13 + a^10 + a^9 + a^6 + a^5 + a^4 + a^2 + a + 1,
 a^15 + a^14 + a^12 + a^6 + a^5 + a^3,
 a^14 + a^13 + a^7 + a^4 + a^3 + 1,
 a^15 + a^14 + a^13 + a^12 + a^9 + a^8 + a^7 + a^5 + a,
 a^15 + a^13 + a^12 + a^10 + a^9 + a^8 + a^7 + a^6 + a^3 + a^2 + a]

### Decoding function

In [26]:
def decode_f(k, r, y, erased_symbols):
    inf_positions, par_positions = get_positions_f(k, r)
    positions = inf_positions + par_positions
    
    t = len(erased_symbols)
    erased_positions = [positions[i] for i in erased_symbols]
    
    syndrome_poly = get_syndrome_poly_f(y, positions, t)
    erasure_locator_poly = get_erasure_locator_poly_f(erased_positions)
    erasure_evaluator_poly = get_erasure_evaluator_poly_f(erasure_locator_poly, syndrome_poly, t)
    
    repaired_code_vector = get_repaired_code_vector_f(y, erasure_locator_poly, erasure_evaluator_poly, erased_symbols, erased_positions)
    
    return repaired_code_vector

## Tests

In [27]:
print(code_vector == repaired_code_vector)
len(list(i for i in range(k + r) if code_vector[i] != repaired_code_vector[i]))

True


0

In [28]:
cases = 100

for c in range(1, cases + 1):
    if c % 10 == 0:
        print("Running test", c)
    
    k = random.randint(100, 200)
    r = random.randint(15, 50)
    t = random.randint(10, r)
    
    payload_data_vector = generate_payload_data_vector_f(k)
    code_vector, positions = encode_f(payload_data_vector, r, verbose=False)
    
    y, erased_symbols, _ = erase_symbols_f(code_vector, positions, t)
    while y == code_vector:
        print("W: y == code_vector, trying again")
        y, erased_symbols, _ = erase_symbols_f(code_vector, positions, t)
    assert y != code_vector
    
    repaired_code_vector = decode_f(k, r, y, erased_symbols)
    
    assert code_vector == repaired_code_vector
print("Tests completed")

Running test 10
Running test 20
Running test 30
Running test 40
Running test 50
Running test 60
Running test 70
Running test 80
Running test 90
Running test 100
Tests completed


# Tests for C code

In [29]:
def symbol_seq_to_struct_init(s, symbol_size, l):
    # s is transposed relative to C code repr
    return "{" + ", ".join(
        "{" + ", ".join(
            str(s[i][j].to_integer()) 
            for i in range(symbol_size)
        ) + "}" 
        for j in range(l)
    ) + "}"
    
def symbols_data_to_symbols(data_name, l):
    return "{" + ", ".join(
        f"{{.data = {data_name}[{i}]}}"
        for i in range(l)
    ) + "}"

## Encoding tests

In [30]:
def generate_encoding_test_code(symbol_size, k, r):
    payload_data_vector = [generate_payload_data_vector_f(k) for _ in range(symbol_size)]
    code_vector = [encode_f(payload_data_vector[i], r, verbose=False)[0] for i in range(symbol_size)]
    rep_symbols = [code_vector[i][k:] for i in range(symbol_size)]
    
    inf_symbols_data = symbol_seq_to_struct_init(payload_data_vector, symbol_size, k)
    rep_symbols_data = symbol_seq_to_struct_init(rep_symbols, symbol_size, r)
    inf_symbols_array = symbols_data_to_symbols("inf_symbols_data", k)
    rep_symbols_array = symbols_data_to_symbols("rep_symbols_data", r)
        
    test_code = f"""{{
    element_t inf_symbols_data[{k}][{symbol_size}] = {inf_symbols_data};
    element_t rep_symbols_data[{r}][{symbol_size}] = {rep_symbols_data};
    symbol_t inf_symbols_array[{k}] = {inf_symbols_array};
    symbol_t rep_symbols_array[{r}] = {rep_symbols_array}; 
    symbol_seq_t inf_symbols = {{.symbol_size = {symbol_size}, .length = {k}, .symbols = inf_symbols_array}};
    symbol_seq_t rep_symbols = {{.symbol_size = {symbol_size}, .length = {r}, .symbols = rep_symbols_array}};

    TEST_WRAPPER(rs, inf_symbols, rep_symbols);
}}"""
    
    print("Test code:\n" + test_code)

In [31]:
# Test 1

symbol_size = 1
k = 22
r = 17

generate_encoding_test_code(symbol_size, k, r)

Test code:
{
    element_t inf_symbols_data[22][1] = {{23463}, {18271}, {63830}, {12679}, {25486}, {25593}, {11151}, {6604}, {18508}, {3787}, {15591}, {63471}, {37059}, {46772}, {58133}, {17827}, {30262}, {54112}, {56282}, {42876}, {32057}, {64947}};
    element_t rep_symbols_data[17][1] = {{44207}, {22087}, {26603}, {55142}, {49663}, {14523}, {49331}, {53134}, {10148}, {61427}, {11786}, {14279}, {11259}, {7497}, {31657}, {53110}, {24002}};
    symbol_t inf_symbols_array[22] = {{.data = inf_symbols_data[0]}, {.data = inf_symbols_data[1]}, {.data = inf_symbols_data[2]}, {.data = inf_symbols_data[3]}, {.data = inf_symbols_data[4]}, {.data = inf_symbols_data[5]}, {.data = inf_symbols_data[6]}, {.data = inf_symbols_data[7]}, {.data = inf_symbols_data[8]}, {.data = inf_symbols_data[9]}, {.data = inf_symbols_data[10]}, {.data = inf_symbols_data[11]}, {.data = inf_symbols_data[12]}, {.data = inf_symbols_data[13]}, {.data = inf_symbols_data[14]}, {.data = inf_symbols_data[15]}, {.data = inf_sy

In [32]:
# Test 2

symbol_size = 2
k = 13
r = 8

generate_encoding_test_code(symbol_size, k, r)

Test code:
{
    element_t inf_symbols_data[13][2] = {{52386, 18372}, {17054, 63597}, {12776, 63070}, {27583, 22037}, {764, 14105}, {27334, 11260}, {55821, 19383}, {4226, 48718}, {31993, 60926}, {5035, 12882}, {57381, 1912}, {28615, 23561}, {38931, 23609}};
    element_t rep_symbols_data[8][2] = {{43618, 45895}, {50876, 41375}, {59615, 31709}, {5306, 5088}, {51195, 25755}, {21369, 14410}, {48312, 20644}, {46844, 22666}};
    symbol_t inf_symbols_array[13] = {{.data = inf_symbols_data[0]}, {.data = inf_symbols_data[1]}, {.data = inf_symbols_data[2]}, {.data = inf_symbols_data[3]}, {.data = inf_symbols_data[4]}, {.data = inf_symbols_data[5]}, {.data = inf_symbols_data[6]}, {.data = inf_symbols_data[7]}, {.data = inf_symbols_data[8]}, {.data = inf_symbols_data[9]}, {.data = inf_symbols_data[10]}, {.data = inf_symbols_data[11]}, {.data = inf_symbols_data[12]}};
    symbol_t rep_symbols_array[8] = {{.data = rep_symbols_data[0]}, {.data = rep_symbols_data[1]}, {.data = rep_symbols_data[2]}, 

In [33]:
# Test 3

symbol_size = 3
k = 7
r = 4

generate_encoding_test_code(symbol_size, k, r)

Test code:
{
    element_t inf_symbols_data[7][3] = {{48683, 58717, 46625}, {24552, 22868, 2893}, {63033, 9253, 29098}, {26837, 13419, 33900}, {40432, 49644, 23866}, {32948, 40685, 50908}, {26532, 29916, 49073}};
    element_t rep_symbols_data[4][3] = {{22971, 42066, 18978}, {38194, 49653, 33954}, {8928, 19722, 46433}, {60326, 44855, 5916}};
    symbol_t inf_symbols_array[7] = {{.data = inf_symbols_data[0]}, {.data = inf_symbols_data[1]}, {.data = inf_symbols_data[2]}, {.data = inf_symbols_data[3]}, {.data = inf_symbols_data[4]}, {.data = inf_symbols_data[5]}, {.data = inf_symbols_data[6]}};
    symbol_t rep_symbols_array[4] = {{.data = rep_symbols_data[0]}, {.data = rep_symbols_data[1]}, {.data = rep_symbols_data[2]}, {.data = rep_symbols_data[3]}}; 
    symbol_seq_t inf_symbols = {.symbol_size = 3, .length = 7, .symbols = inf_symbols_array};
    symbol_seq_t rep_symbols = {.symbol_size = 3, .length = 4, .symbols = rep_symbols_array};

    TEST_WRAPPER(rs, inf_symbols, rep_symbols);
}

## Erasure correction tests

In [50]:
def generate_erasure_correction_test_code(symbol_size, k, r, t):
    payload_data_vector = [generate_payload_data_vector_f(k) for _ in range(symbol_size)]
    code_vector = [encode_f(payload_data_vector[i], r, verbose=False)[0] for i in range(symbol_size)]
    erased_indices = random.sample(range(k + r), t)
    
    rcv_symbols = [code_vector[i].copy() for i in range(symbol_size)]
    # for erased_idx in erased_indices:
    #     for i in range(symbol_size):
    #         rcv_symbols[i][erased_idx] *= 0
            
    rcv_symbols_data = symbol_seq_to_struct_init(rcv_symbols, symbol_size, k + r)
    rcv_symbols_array = symbols_data_to_symbols("rcv_symbols_data", k + r)
    erased_indices_str = "{" + ", ".join(map(str, erased_indices)) + "}"
        
    test_code = f"""{{
    uint16_t k = {k};
    uint16_t r = {r};
    uint16_t t = {t};
    element_t rcv_symbols_data[{k + r}][{symbol_size}] = {rcv_symbols_data};
    symbol_t rcv_symbols_array[{k + r}] = {rcv_symbols_array}; 
    symbol_seq_t rcv_symbols = {{.symbol_size = {symbol_size}, .length = {k + r}, .symbols = rcv_symbols_array}};
    uint16_t erased_indices[{t}] = {erased_indices_str};

    TEST_WRAPPER(rs, k, r, rcv_symbols, erased_indices, t);
}}"""
    
    print("Test code:\n" + test_code)

In [51]:
# Test 1

symbol_size = 1
k = 24
r = 15
t = 12

generate_erasure_correction_test_code(symbol_size, k, r, t)

Test code:
{
    uint16_t k = 24;
    uint16_t r = 15;
    uint16_t t = 12;
    element_t rcv_symbols_data[39][1] = {{45100}, {44816}, {14317}, {14605}, {39619}, {14144}, {33136}, {47610}, {11155}, {6240}, {9065}, {27926}, {31952}, {44615}, {44897}, {58344}, {3297}, {21777}, {36711}, {5053}, {44272}, {24319}, {21107}, {17830}, {48152}, {46421}, {16974}, {26755}, {40689}, {2995}, {20301}, {34240}, {58232}, {12357}, {18371}, {41310}, {11768}, {54399}, {63455}};
    symbol_t rcv_symbols_array[39] = {{.data = rcv_symbols_data[0]}, {.data = rcv_symbols_data[1]}, {.data = rcv_symbols_data[2]}, {.data = rcv_symbols_data[3]}, {.data = rcv_symbols_data[4]}, {.data = rcv_symbols_data[5]}, {.data = rcv_symbols_data[6]}, {.data = rcv_symbols_data[7]}, {.data = rcv_symbols_data[8]}, {.data = rcv_symbols_data[9]}, {.data = rcv_symbols_data[10]}, {.data = rcv_symbols_data[11]}, {.data = rcv_symbols_data[12]}, {.data = rcv_symbols_data[13]}, {.data = rcv_symbols_data[14]}, {.data = rcv_symbols_data[15

In [52]:
# Test 2

symbol_size = 2
k = 11
r = 9
t = 9

generate_erasure_correction_test_code(symbol_size, k, r, t)

Test code:
{
    uint16_t k = 11;
    uint16_t r = 9;
    uint16_t t = 9;
    element_t rcv_symbols_data[20][2] = {{21077, 59249}, {51477, 43149}, {50557, 27336}, {40204, 52243}, {53043, 2522}, {62456, 19727}, {32103, 6835}, {32456, 22811}, {23396, 5399}, {3297, 10260}, {49033, 62857}, {17411, 21680}, {58842, 52508}, {34182, 23221}, {40140, 17215}, {50916, 50726}, {21565, 5511}, {48243, 12175}, {49927, 15781}, {16743, 26493}};
    symbol_t rcv_symbols_array[20] = {{.data = rcv_symbols_data[0]}, {.data = rcv_symbols_data[1]}, {.data = rcv_symbols_data[2]}, {.data = rcv_symbols_data[3]}, {.data = rcv_symbols_data[4]}, {.data = rcv_symbols_data[5]}, {.data = rcv_symbols_data[6]}, {.data = rcv_symbols_data[7]}, {.data = rcv_symbols_data[8]}, {.data = rcv_symbols_data[9]}, {.data = rcv_symbols_data[10]}, {.data = rcv_symbols_data[11]}, {.data = rcv_symbols_data[12]}, {.data = rcv_symbols_data[13]}, {.data = rcv_symbols_data[14]}, {.data = rcv_symbols_data[15]}, {.data = rcv_symbols_data[16]

In [53]:
# Test 3

symbol_size = 3
k = 9
r = 6
t = 6

generate_erasure_correction_test_code(symbol_size, k, r, t)

Test code:
{
    uint16_t k = 9;
    uint16_t r = 6;
    uint16_t t = 6;
    element_t rcv_symbols_data[15][3] = {{30234, 54688, 40968}, {62001, 57116, 53248}, {59608, 40439, 47910}, {40449, 44643, 1356}, {41281, 61935, 24043}, {20928, 64334, 28787}, {551, 33036, 16640}, {54685, 57197, 62733}, {27135, 7861, 42303}, {26749, 9013, 13320}, {15209, 7220, 20689}, {62126, 34159, 34348}, {22415, 10559, 35455}, {23533, 63689, 15527}, {4590, 6341, 42725}};
    symbol_t rcv_symbols_array[15] = {{.data = rcv_symbols_data[0]}, {.data = rcv_symbols_data[1]}, {.data = rcv_symbols_data[2]}, {.data = rcv_symbols_data[3]}, {.data = rcv_symbols_data[4]}, {.data = rcv_symbols_data[5]}, {.data = rcv_symbols_data[6]}, {.data = rcv_symbols_data[7]}, {.data = rcv_symbols_data[8]}, {.data = rcv_symbols_data[9]}, {.data = rcv_symbols_data[10]}, {.data = rcv_symbols_data[11]}, {.data = rcv_symbols_data[12]}, {.data = rcv_symbols_data[13]}, {.data = rcv_symbols_data[14]}}; 
    symbol_seq_t rcv_symbols = {.symbo